# Unit 8 SQL Case Study Part 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook
for the following questions.

QUESTIONS:
 - Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!
 - Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
 - Q12: Find the facilities with their usage by member, but not guests
 - Q13: Find the facilities usage by month, but not guests

In [1]:
# Import packages and functions
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd

In [2]:
# Create engine, connecting to the file provided
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

print(inspect(engine).get_table_names())

def collect_data(query, con):
    """
    Accepts a query and connection to return a DataFrame with labeled columns that represents the table.
    """
    rs = con.execute(query)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    return df

['Bookings', 'Facilities', 'Members']


In [2]:
# Context manager to collect data into DataFrames
#with engine.connect() as con:
#    members_df = collect_data("SELECT * FROM Members", con)
#    bookings_df = collect_data("SELECT * From Bookings", con)
#    facilities_df = collect_data("SELECT * From Facilities", con)

#print(members_df.head())
#print(bookings_df.head())
#print(facilities_df.head())

#print(con.closed)

['Bookings', 'Facilities', 'Members']
   memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-02 12:02:05  
2    555-555-5555                2012-07-02 12:08:23  
3  (844) 693-0723                2012-07-03 09:32:15  
4  (833) 942-4710             1  2012-07-03 10:25:05  
   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00     

## Q10
Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [3]:
# Revenue for each facility = count bookings * slots * cost_type
# Ordered by revenue
# Output: facility_name, revenue
with engine.connect() as con:
    q10_query = """SELECT Facilities.name AS facility_name,
    SUM(CASE WHEN (Bookings.memid = 0) THEN (Bookings.slots * Facilities.guestcost)
        ELSE (Bookings.slots * Facilities.membercost)
        END) AS revenue
    FROM Bookings
    LEFT JOIN Facilities
    ON Bookings.facid = Facilities.facid
    GROUP BY Facilities.name
    HAVING revenue < 1000
    ORDER BY revenue;"""
    q10_df = collect_data(q10_query, con)

print(q10_df)

   facility_name  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


## Q11
Produce a report of members and who recommended them in alphabetic surname,firstname order

In [4]:
q11_query = """SELECT m1.surname, m1.firstname,
m2.surname AS rec_by_surname, m2.firstname AS rec_by_firstname
FROM Members AS m1
INNER JOIN Members as m2
ON m1.recommendedby = m2.memid
ORDER BY m1.surname, m1.firstname;"""

q11_df = pd.read_sql_query(q11_query, engine)
print(q11_df)
print(len(q11_df))

              surname  firstname rec_by_surname rec_by_firstname
0               Bader   Florence       Stibbons           Ponder
1               Baker       Anne       Stibbons           Ponder
2               Baker    Timothy        Farrell           Jemima
3              Boothe        Tim         Rownam              Tim
4             Butters     Gerald          Smith           Darren
5              Coplin       Joan          Baker          Timothy
6             Crumpet      Erica          Smith            Tracy
7                Dare      Nancy       Joplette           Janice
8             Genting    Matthew        Butters           Gerald
9                Hunt       John        Purview        Millicent
10              Jones      David       Joplette           Janice
11              Jones    Douglas          Jones            David
12           Joplette     Janice          Smith           Darren
13          Mackenzie       Anna          Smith           Darren
14               Owen    

The above result does not include records that had missing values for recommendedby.

## Q12
Find the facilities with their usage by member, but not guests

In [5]:
q12_query = """SELECT Facilities.name, COUNT(memid) AS uses_by_member
FROM Bookings
LEFT JOIN Facilities
ON Bookings.facid = Facilities.facid
WHERE memid <> 0
GROUP BY Bookings.facid, Facilities.name
ORDER BY Bookings.facid;"""

q12_df = pd.read_sql_query(q12_query, engine)
print(q12_df)

              name  uses_by_member
0   Tennis Court 1             308
1   Tennis Court 2             276
2  Badminton Court             344
3     Table Tennis             385
4   Massage Room 1             421
5   Massage Room 2              27
6     Squash Court             195
7    Snooker Table             421
8       Pool Table             783


## Q13
Find the facilities usage by month, but not guests

In [6]:
q13_query = """SELECT Facilities.name AS facility_name,
    CASE WHEN (Bookings.starttime LIKE '2012-07%') THEN 'July'
    WHEN (Bookings.starttime LIKE '2012-08%') THEN 'August'
    WHEN (Bookings.starttime LIKE '2012-09%') THEN 'September' END AS month,
    COUNT(bookid) AS total_books
FROM Bookings
RIGHT JOIN Facilities
ON Bookings.facid = Facilities.facid
GROUP BY facility_name, month;"""

q13_df = pd.read_sql_query(q13_query, engine)
print(q13_df)

      facility_name      month  total_books
0   Badminton Court     August          146
1   Badminton Court       July           56
2   Badminton Court  September          181
3    Massage Room 1     August          224
4    Massage Room 1       July          123
5    Massage Room 1  September          282
6    Massage Room 2     August           40
7    Massage Room 2       July           12
8    Massage Room 2  September           59
9        Pool Table     August          291
10       Pool Table       July          110
11       Pool Table  September          435
12    Snooker Table     August          159
13    Snooker Table       July           75
14    Snooker Table  September          210
15     Squash Court     August          170
16     Squash Court       July           75
17     Squash Court  September          195
18     Table Tennis     August          147
19     Table Tennis       July           51
20     Table Tennis  September          205
21   Tennis Court 1     August  

Adding this part to the query would exclude guests, if that is what the question is asking:
```
WHERE Bookings.memid <> 0
```